In [1]:
import sklearn
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn
from collections import defaultdict
from sklearn.metrics import classification_report

In [4]:
results_all = pd.read_pickle("Loop3_results_dict_of_dfs.pickle")

In [6]:
# results_all

### Make metrics dataframe for GPT definitions

In [17]:
metrics_df = pd.DataFrame()

definitions = ["results_GPTdef10", "results_GPTdef1"]
experts = ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9","EXP_10","EXP_11"]

for dataset in results_all["sexism"].keys():
    for definition in definitions:
        for expert in experts: 
            for i in results_all["sexism"][dataset][definition][:2]:
                result_dict = eval(i)
#                 print(result_dict)
                try:
                    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict[dataset][expert].values()]
                    labels = ["non-sexist" if i == "can’t say" or i == "can't say" else i for i in labels] 
                    truth = results_all["sexism"][dataset]["sexism"].tolist()
                    scores = classification_report(truth, labels, output_dict=True,zero_division=0)

                    temp_dict = pd.DataFrame()

                    temp_dict["definition"] = [definition]
                    temp_dict["expert"] = expert
                    temp_dict["dataset"] = dataset
                    temp_dict["scores"] = [scores]

                    metrics_df = pd.concat([metrics_df, temp_dict])
                except:
                    print("skip")

skip
skip


### Add the participant definitions to this dataframe

In [18]:
for dataset in results_all["sexism"].keys():
    for definition in ["results"]:
        for expert in experts: 
            for i in results_all["sexism"][dataset][definition][:2]:
                result_dict = eval(i)
#                 print(result_dict)
                try:
                    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["GPT4o"][expert].values()]
                    labels = ["non-sexist" if i == "can’t say" or i == "can't say" else i for i in labels] 
                    truth = results_all["sexism"][dataset]["sexism"].tolist()
                    scores = classification_report(truth, labels, output_dict=True)

                    temp_dict = pd.DataFrame()

                    temp_dict["definition"] = [definition]
                    temp_dict["expert"] = expert
                    temp_dict["dataset"] = dataset
                    temp_dict["scores"] = [scores]

                    metrics_df = pd.concat([metrics_df, temp_dict])
                except:
                    print("skip")

In [19]:
metrics_df["definition"] = metrics_df["definition"].replace("results_GPTdef10", "hybrid")
metrics_df["definition"] = metrics_df["definition"].replace("results_GPTdef1", "GPT")
metrics_df["definition"] = metrics_df["definition"].replace("results", "participant")

In [20]:
for i in metrics_df["scores"][:1]:
    print(i)

{'non-sexist': {'precision': 0.9333333333333333, 'recall': 0.7608695652173914, 'f1-score': 0.8383233532934132, 'support': 276}, 'sexist': {'precision': 0.76, 'recall': 0.9330357142857143, 'f1-score': 0.8376753507014029, 'support': 224}, 'accuracy': 0.838, 'macro avg': {'precision': 0.8466666666666667, 'recall': 0.8469526397515528, 'f1-score': 0.837999351997408, 'support': 500}, 'weighted avg': {'precision': 0.8556800000000001, 'recall': 0.838, 'f1-score': 0.8380330481321926, 'support': 500}}


### add individual metrics to dataframe

In [21]:
F1s = []
in_sex = []
in_non = []

for index, row in metrics_df.iterrows():
    F1 = dict(row["scores"])["macro avg"]["f1-score"]
    class_sexism_f1 = dict(row["scores"])["sexist"]["f1-score"]
    non_sex = dict(row["scores"])["non-sexist"]["f1-score"]
    F1s.append(F1)
    in_sex.append(class_sexism_f1)
    in_non.append(non_sex)
    
#     print(F1)
#     print(in_sex)

metrics_df["F1"] = F1s
metrics_df["sexist_F1"] = in_sex
metrics_df["non_sexist_F1"] = in_non
metrics_df

,definition,expert,dataset,scores,F1,sexist_F1,non_sexist_F1
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_3,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.811852,0.817121,0.806584
...,...,...,...,...,...,...,...
0,participant,EXP_9,hatecheck,{'non-sexist': {'precision': 0.267857142857142...,0.483775,0.631415,0.336134
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022
0,participant,EXP_11,hatecheck,{'non-sexist': {'precision': 0.271551724137931...,0.484414,0.623622,0.345205


### clean data of duplicates and participants who didn't do it

In [23]:
clean_data = metrics_df[metrics_df[["definition", "expert", "dataset", "F1", "sexist_F1", "non_sexist_F1"]].duplicated()]

In [24]:
clean_data

,definition,expert,dataset,scores,F1,sexist_F1,non_sexist_F1
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_3,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0.811852,0.817121,0.806584
0,hybrid,EXP_4,in_domain,{'non-sexist': {'precision': 0.938095238095238...,0.815856,0.821012,0.810700
0,hybrid,EXP_5,in_domain,{'non-sexist': {'precision': 0.921739130434782...,0.835976,0.834008,0.837945
...,...,...,...,...,...,...,...
0,participant,EXP_7,hatecheck,{'non-sexist': {'precision': 0.266375545851528...,0.480421,0.623824,0.337017
0,participant,EXP_8,hatecheck,{'non-sexist': {'precision': 0.269911504424778...,0.485049,0.630265,0.339833
0,participant,EXP_9,hatecheck,{'non-sexist': {'precision': 0.267857142857142...,0.483775,0.631415,0.336134
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022


In [25]:
# for i in range(12):
#     filtered_data = clean_data[clean_data["expert"] == f"EXP_{i}"]
#     print(filtered_data)



In [29]:
clean_data["dataset"] = clean_data["dataset"].replace("in_domain", "CallMeSexist")
clean_data["dataset"] = clean_data["dataset"].replace("out_of_domain_3", "EDOS")
clean_data["dataset"] = clean_data["dataset"].replace("out_of_domain_2", "RedditGuest")
clean_data["dataset"] = clean_data["dataset"].replace("out_of_domain", "EXIST")


In [30]:
clean_data = clean_data[clean_data.expert != "EXP_8"]
clean_data = clean_data[clean_data.expert != "EXP_7"]
clean_data

clean_data.to_pickle('cleaned_data.pkl')


In [33]:
clean_data

,definition,expert,dataset,scores,F1,sexist_F1,non_sexist_F1
0,hybrid,EXP_1,CallMeSexist,{'non-sexist': {'precision': 0.933333333333333...,0.837999,0.837675,0.838323
0,hybrid,EXP_2,CallMeSexist,{'non-sexist': {'precision': 0.913793103448275...,0.831957,0.829268,0.834646
0,hybrid,EXP_3,CallMeSexist,{'non-sexist': {'precision': 0.933333333333333...,0.811852,0.817121,0.806584
0,hybrid,EXP_4,CallMeSexist,{'non-sexist': {'precision': 0.938095238095238...,0.815856,0.821012,0.810700
0,hybrid,EXP_5,CallMeSexist,{'non-sexist': {'precision': 0.921739130434782...,0.835976,0.834008,0.837945
...,...,...,...,...,...,...,...
0,participant,EXP_5,hatecheck,{'non-sexist': {'precision': 0.265384615384615...,0.465523,0.579901,0.351145
0,participant,EXP_6,hatecheck,{'non-sexist': {'precision': 0.275229357798165...,0.492974,0.644068,0.341880
0,participant,EXP_9,hatecheck,{'non-sexist': {'precision': 0.267857142857142...,0.483775,0.631415,0.336134
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0.476382,0.661743,0.291022


### Show cleaned data

In [27]:
clean_data.groupby("definition")["F1"].std()

definition
GPT            0.113551
hybrid         0.120875
participant    0.154190
Name: F1, dtype: float64

In [37]:
clean_data.groupby(["definition"])["non_sexist_F1"].mean()

definition
GPT            0.837355
hybrid         0.834876
participant    0.557013
Name: non_sexist_F1, dtype: float64

In [36]:
clean_data.groupby(["dataset", "definition"])["non_sexist_F1"].mean()

dataset       definition 
CallMeSexist  GPT            0.813739
              hybrid         0.812609
              participant    0.823685
EDOS          GPT            0.807345
              hybrid         0.811304
              participant    0.564190
EXIST         GPT            0.809151
              hybrid         0.811036
              participant    0.475008
RedditGuest   GPT            0.894011
              hybrid         0.888337
              participant    0.583967
hatecheck     GPT            0.859397
              hybrid         0.851093
              participant    0.338215
Name: non_sexist_F1, dtype: float64

In [38]:
clean_data.groupby(["dataset", "definition"])["sexist_F1"].mean()

dataset       definition 
CallMeSexist  GPT            0.819466
              hybrid         0.819820
              participant    0.825639
EDOS          GPT            0.622110
              hybrid         0.627971
              participant    0.330570
EXIST         GPT            0.740047
              hybrid         0.749110
              participant    0.509250
RedditGuest   GPT            0.558176
              hybrid         0.544955
              participant    0.203922
hatecheck     GPT            0.954368
              hybrid         0.951913
              participant    0.635904
Name: sexist_F1, dtype: float64

In [45]:
clean_data.groupby(["expert", "definition"])["F1", "sexist_F1", "non_sexist_F1"].mean()

<ipython-input-45-e6aef083c138>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  clean_data.groupby(["expert", "definition"])["F1", "sexist_F1", "non_sexist_F1"].mean()


F1  sexist_F1  non_sexist_F1
expert definition                                     
EXP_1  GPT          0.744246   0.744452       0.841634
       hybrid       0.741641   0.741673       0.837508
       participant  0.528762   0.508039       0.549485
EXP_10 GPT          0.796137   0.742001       0.850272
       hybrid       0.784107   0.736482       0.831731
       participant  0.505567   0.506695       0.504438
EXP_11 GPT          0.740651   0.740151       0.839322
       hybrid       0.723369   0.732582       0.808814
       participant  0.533880   0.496867       0.570893
EXP_2  GPT          0.745096   0.743421       0.845247
       hybrid       0.785828   0.731277       0.840378
       participant  0.531685   0.504144       0.559226
EXP_3  GPT          0.784345   0.736838       0.831852
       hybrid       0.734729   0.736772       0.827774
       participant  0.532776   0.502817       0.562735
EXP_4  GPT          0.784440   0.734373       0.834506
       hybrid       0.788122   0.741242       0.835003
       participant  0.531668   0.506638       0.556697
EXP_5  GPT          0.737392   0.738024       0.830754
       hybrid       0.756336   0.749429       0.859477
       participant  0.539335   0.480943       0.597728
EXP_6  GPT          0.777784   0.731774       0.823794
       hybrid       0.739151   0.739088       0.835664
       participant  0.527122   0.503635       0.550610
EXP_9  GPT          0.787788   0.737329       0.838246
       hybrid       0.788885   0.740239       0.837532
       participant  0.530519   0.499734       0.561304

In [51]:
from tabulate import tabulate

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(tabulate(clean_data.groupby(["expert", "definition"])["F1", "sexist_F1", "non_sexist_F1"].mean()))

-------------------------  --------  --------  --------
('EXP_1', 'GPT')           0.744246  0.744452  0.841634
('EXP_1', 'hybrid')        0.741641  0.741673  0.837508
('EXP_1', 'participant')   0.528762  0.508039  0.549485
('EXP_10', 'GPT')          0.796137  0.742001  0.850272
('EXP_10', 'hybrid')       0.784107  0.736482  0.831731
('EXP_10', 'participant')  0.505567  0.506695  0.504438
('EXP_11', 'GPT')          0.740651  0.740151  0.839322
('EXP_11', 'hybrid')       0.723369  0.732582  0.808814
('EXP_11', 'participant')  0.53388   0.496867  0.570893
('EXP_2', 'GPT')           0.745096  0.743421  0.845247
('EXP_2', 'hybrid')        0.785828  0.731277  0.840378
('EXP_2', 'participant')   0.531685  0.504144  0.559226
('EXP_3', 'GPT')           0.784345  0.736838  0.831852
('EXP_3', 'hybrid')        0.734729  0.736772  0.827774
('EXP_3', 'participant')   0.532776  0.502817  0.562735
('EXP_4', 'GPT')           0.78444   0.734373  0.834506
('EXP_4', 'hybrid')        0.788122  0.741242  0

<ipython-input-51-231fdc0680bd>:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  print(tabulate(clean_data.groupby(["expert", "definition"])["F1", "sexist_F1", "non_sexist_F1"].mean()))


In [44]:
dataset       definition 
CallMeSexist  GPT            0.814498
              hybrid         0.822511
              participant    0.826991
EDOS          GPT            0.711774
              hybrid         0.726304
              participant    0.450252
EXIST         GPT            0.778799
              hybrid         0.776701
              participant    0.494881
RedditGuest   GPT            0.619981
              hybrid         0.581450
              participant    0.397389
hatecheck     GPT            0.903492
              hybrid         0.903530
              participant    0.488963
Name: F1, dtype: float64

SyntaxError: invalid syntax (<ipython-input-44-d67305dbb410>, line 1)

In [85]:
# Convert the 'expert' column to a categorical type with the desired order
clean_data["expert"] = pd.Categorical(
    clean_data["expert"],
    categories=sorted(clean_data["expert"].unique(), key=lambda x: int(x.split('_')[1])),
    ordered=True
)

clean_data.groupby(["expert", "definition"])["F1"].mean().sort_index()


expert  definition 
EXP_1   GPT            0.744246
        hybrid         0.741641
        participant    0.528762
EXP_2   GPT            0.745096
        hybrid         0.785828
        participant    0.531685
EXP_3   GPT            0.784345
        hybrid         0.734729
        participant    0.532776
EXP_4   GPT            0.784440
        hybrid         0.788122
        participant    0.531668
EXP_5   GPT            0.737392
        hybrid         0.756336
        participant    0.539335
EXP_6   GPT            0.777784
        hybrid         0.739151
        participant    0.527122
EXP_9   GPT            0.787788
        hybrid         0.788885
        participant    0.530519
EXP_10  GPT            0.796137
        hybrid         0.784107
        participant    0.505567
EXP_11  GPT            0.740651
        hybrid         0.723369
        participant    0.533880
Name: F1, dtype: float64

In [81]:
metrics_df["F1"] = 0

for index, row in metrics_df.iterrows():
    metrics_df[index, "F1"] = dict(row["scores"])["macro avg"]["f1-score"]
# #     metrics_df.loc[index, "f1"] = row["scores"]["macro avg"]["f1-score"]
#     print(row)
    
metrics_df

,definition,expert,dataset,scores,F1,sexist_F1,non_sexist_F1,"(0, F1)"
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0,0.837675,0.838323,0.484414
0,hybrid,EXP_1,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0,0.837675,0.838323,0.484414
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0,0.829268,0.834646,0.484414
0,hybrid,EXP_2,in_domain,{'non-sexist': {'precision': 0.913793103448275...,0,0.829268,0.834646,0.484414
0,hybrid,EXP_3,in_domain,{'non-sexist': {'precision': 0.933333333333333...,0,0.817121,0.806584,0.484414
...,...,...,...,...,...,...,...,...
0,participant,EXP_9,hatecheck,{'non-sexist': {'precision': 0.267857142857142...,0,0.631415,0.336134,0.484414
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0,0.661743,0.291022,0.484414
0,participant,EXP_10,hatecheck,{'non-sexist': {'precision': 0.247368421052631...,0,0.661743,0.291022,0.484414
0,participant,EXP_11,hatecheck,{'non-sexist': {'precision': 0.271551724137931...,0,0.623622,0.345205,0.484414


In [82]:
datasets = ['out_of_domain', 
             'out_of_domain_2', 'out_of_domain_3', #'out_of_domain_4', 
             'hatecheck']

In [48]:
from collections import defaultdict
from sklearn.metrics import classification_report

In [49]:
import warnings
warnings.filterwarnings('ignore') 

# for i in results_all["sexism"]['hatecheck']["results_GPTdef10"]:
#     result_dict = eval(i)
#     print(result_dict)
for dataset in datasets:
    print(dataset)
    for i in results_all["sexism"][dataset]["results_GPTdef10"][:1]:
        result_dict = eval(i)
        labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict[dataset]["EXP_3"].values()]
        labels = ["non-sexist" if i == "can’t say" or i == "can't say" else i for i in labels] 
        #         print(labels)
        truth = results_all["sexism"][dataset]["sexism"].tolist()
        print(classification_report(truth, labels))
        print()
        print()

out_of_domain
              precision    recall  f1-score   support

  non-sexist       0.76      0.89      0.82       262
      sexist       0.85      0.70      0.76       238

    accuracy                           0.80       500
   macro avg       0.81      0.79      0.79       500
weighted avg       0.80      0.80      0.79       500



out_of_domain_2
              precision    recall  f1-score   support

  non-sexist       0.98      0.81      0.88       435
      sexist       0.40      0.85      0.54        65
   sexist

2       0.00      0.00      0.00         0

    accuracy                           0.81       500
   macro avg       0.46      0.55      0.48       500
weighted avg       0.90      0.81      0.84       500



out_of_domain_3
              precision    recall  f1-score   support

  non-sexist       0.96      0.68      0.80       384
      sexist       0.46      0.91      0.61       116

    accuracy                           0.73       500
   macro avg       0.71 

In [50]:
import warnings
warnings.filterwarnings('ignore') 

for i in results_all["sexism"]['hatecheck']["results_GPTdef10"]:
    result_dict = eval(i)
#     print(result_dict)
for participant in ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]:
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["hatecheck"][participant].values()]
    labels = ["non-sexist" if i == "can’t say" else i for i in labels] 
    #         print(labels)
    truth = results_all["sexism"]['hatecheck']["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

  non-sexist       0.89      0.77      0.83       133
      sexist       0.92      0.96      0.94       367

    accuracy                           0.91       500
   macro avg       0.90      0.87      0.89       500
weighted avg       0.91      0.91      0.91       500


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.86      0.81      0.83       133
      sexist       0.93      0.95      0.94       367

    accuracy                           0.91       500
   macro avg       0.90      0.88      0.89       500
weighted avg       0.91      0.91      0.91       500


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.95      0.74      0.83       133
      sexist       0.91      0.99      0.95       367

    accuracy                           0.92       500
   macro avg       0.93      0.86      0.89       500
weighted avg       0.92      0.92      0.92       500


In [51]:
import warnings
warnings.filterwarnings('ignore') 

for i in results_all["sexism"]['out_of_domain_3']["results_GPTdef10"]:
    result_dict = eval(i)
#     print(result_dict)
for participant in ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]:
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["out_of_domain_3"][participant].values()]
    labels = ["non-sexist" if i == "can’t say" else i for i in labels] 
    #         print(labels)
    truth = results_all["sexism"]['out_of_domain_3']["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

  non-sexist       0.96      0.71      0.82       384
      sexist       0.49      0.91      0.63       116

    accuracy                           0.76       500
   macro avg       0.72      0.81      0.73       500
weighted avg       0.85      0.76      0.78       500


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.95      0.76      0.84       384
      sexist       0.52      0.87      0.65       116

    accuracy                           0.78       500
   macro avg       0.73      0.81      0.75       500
weighted avg       0.85      0.78      0.80       500


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.96      0.68      0.80       384
      sexist       0.46      0.91      0.61       116

    accuracy                           0.73       500
   macro avg       0.71      0.79      0.70       500
weighted avg       0.84      0.73      0.75       500


In [52]:
import warnings
warnings.filterwarnings('ignore') 

for i in results_all["sexism"]['out_of_domain_2']["results_GPTdef10"]:
    result_dict = eval(i)
#     print(result_dict)
for participant in ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]:
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["out_of_domain_2"][participant].values()]
    labels = ["non-sexist" if i == "can’t say" else i for i in labels] 
    #         print(labels)
    truth = results_all["sexism"]['out_of_domain_2']["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

  non-sexist       0.97      0.82      0.89       435
      sexist       0.41      0.83      0.55        65
   sexist

1       0.00      0.00      0.00         0

    accuracy                           0.82       500
   macro avg       0.46      0.55      0.48       500
weighted avg       0.90      0.82      0.85       500


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.96      0.84      0.90       435
      sexist       0.42      0.78      0.55        65

    accuracy                           0.83       500
   macro avg       0.69      0.81      0.72       500
weighted avg       0.89      0.83      0.85       500


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.98      0.81      0.88       435
      sexist       0.40      0.85      0.54        65
   sexist

2       0.00      0.00      0.00         0

    accuracy                           0.81       500


In [53]:
import warnings
warnings.filterwarnings('ignore') 

for i in results_all["sexism"]['out_of_domain']["results_GPTdef10"]:
    result_dict = eval(i)
#     print(result_dict)
for participant in ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]:
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["out_of_domain"][participant].values()]
    labels = ["non-sexist" if i == "can’t say" else i for i in labels] 
    #         print(labels)
    truth = results_all["sexism"]['out_of_domain']["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

   can't say       0.00      0.00      0.00         0
  non-sexist       0.74      0.89      0.81       262
      sexist       0.85      0.66      0.75       238

    accuracy                           0.78       500
   macro avg       0.53      0.52      0.52       500
weighted avg       0.79      0.78      0.78       500


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.70      0.92      0.80       262
      sexist       0.86      0.58      0.69       238

    accuracy                           0.75       500
   macro avg       0.78      0.75      0.74       500
weighted avg       0.78      0.75      0.75       500


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.76      0.89      0.82       262
      sexist       0.85      0.70      0.76       238

    accuracy                           0.80       500
   macro avg       0.81      0.79      0.79       500


In [54]:
import warnings
warnings.filterwarnings('ignore') 

for i in results_all["sexism"]['in_domain']["results_GPTdef10"]:
    result_dict = eval(i)
#     print(result_dict)
for participant in ["EXP_1", "EXP_2", "EXP_3", "EXP_4", "EXP_5", "EXP_6", "EXP_7", "EXP_8", "EXP_9", "EXP_10", "EXP_11"]:
    print(participant)
    labels = [i.split(")")[1].strip().split(".")[0] for i in result_dict["in_domain"][participant].values()]
    labels = ["non-sexist" if i == "can’t say" else i for i in labels] 
    #         print(labels)
    truth = results_all["sexism"]['in_domain']["sexism"].tolist()
    print(classification_report(truth, labels))
    print()

EXP_1
              precision    recall  f1-score   support

  non-sexist       0.93      0.76      0.84       276
      sexist       0.76      0.93      0.84       224

    accuracy                           0.84       500
   macro avg       0.85      0.85      0.84       500
weighted avg       0.86      0.84      0.84       500


EXP_2
              precision    recall  f1-score   support

  non-sexist       0.91      0.77      0.83       276
      sexist       0.76      0.91      0.83       224

    accuracy                           0.83       500
   macro avg       0.84      0.84      0.83       500
weighted avg       0.85      0.83      0.83       500


EXP_3
              precision    recall  f1-score   support

  non-sexist       0.93      0.71      0.81       276
      sexist       0.72      0.94      0.82       224

    accuracy                           0.81       500
   macro avg       0.83      0.82      0.81       500
weighted avg       0.84      0.81      0.81       500
